In [5]:
import tweepy

In [2]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAABtNcAEAAAAA1JW%2FL0O4v0BAn87kE8tp0vCCXYc%3Db8OQFJeRCn3tICEvi6TNC1gS6aMPvvB8ZM18ZyyI8bKytuZaba"

In [3]:
class TwitterIngest(tweepy.StreamingClient):
    def on_tweet(self, tweet):
        print(tweet)

In [4]:
stream = TwitterIngest(bearer_token)
stream.add_rules(tweepy.StreamRule("ukraine"))

Response(data=None, includes={}, errors=[{'value': 'ukraine', 'id': '1520576950833659904', 'title': 'DuplicateRule', 'type': 'https://api.twitter.com/2/problems/duplicate-rules'}], meta={'sent': '2022-05-01T19:17:00.954Z', 'summary': {'created': 0, 'not_created': 1, 'valid': 0, 'invalid': 1}})

In [34]:
stream.sample()

Stream encountered an exception
Traceback (most recent call last):
  File "/home/jklasa/anaconda3/envs/heartbeat/lib/python3.9/site-packages/tweepy/streaming.py", line 91, in _connect
    self.on_data(line)
  File "/home/jklasa/anaconda3/envs/heartbeat/lib/python3.9/site-packages/tweepy/streaming.py", line 870, in on_data
    self.on_tweet(tweet)
  File "/tmp/ipykernel_966/904131402.py", line 3, in on_tweet
    print(tweet._fields)
  File "/home/jklasa/anaconda3/envs/heartbeat/lib/python3.9/site-packages/tweepy/mixins.py", line 35, in __getattr__
    raise AttributeError from None
AttributeError


In [8]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [6]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [9]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [11]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) Negative 0.7236
2) Neutral 0.2287
3) Positive 0.0477
